In [13]:
import pandas as pd
import numpy as np

In [14]:
from cycling_manager.sequences import *
from cycling_manager.preprocess import *
from cycling_manager.model import *

In [118]:
merged_df = preprocess(get_data())

merged_df = merged_df[merged_df['year']> 2012]

full df shape (603619, 25)
full df shape (603619, 25)
Score vertical meters - model: DecisionTreeRegressor() score: 0.9995663416590993
Score vertical meters - model: DecisionTreeRegressor() score: 0.9999242401560309
Score vertical meters - model: KNeighborsClassifier() score: 0.9940733372706267


In [108]:
merged_df.describe()

,year,result,gc,distance,points,Avg. speed winner:,Distance:,ProfileScore:,Vert. meters:,Startlist quality score:,adjusted_points,Race category:,Points scale:,Arrival:,Won how:,parcours_type_num,gt_binary,result_bin,types_bin,icon_bin
count,289240.000000,289240.000000,222966.000000,289212.000000,289240.000000,289239.000000,289240.000000,214663.000000,214663.000000,289240.000000,2.117660e+05,0.0,0.0,0.0,0.0,273064.000000,289240.000000,289240.000000,289240.000000,289240.000000
mean,2017.032810,67.317027,73.172811,159.724697,4.910766,41.796714,159.709235,121.637473,2230.155385,783.942525,3.837118e+05,NaN,NaN,NaN,NaN,2.598863,0.351542,0.203713,0.132132,0.110282
std,2.786694,48.420232,48.369252,58.229730,11.547833,4.801665,58.248113,108.700382,1310.795012,463.174944,1.688216e+06,NaN,NaN,NaN,NaN,1.399827,0.477453,0.402759,0.338635,0.313242
min,2013.000000,1.000000,1.000000,0.600000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.000000,26.000000,31.000000,148.100000,0.000000,38.960000,148.100000,31.000000,1279.000000,395.000000,0.000000e+00,NaN,NaN,NaN,NaN,2.000000,0.000000,0.000000,0.000000,0.000000
50%,2017.000000,60.000000,68.000000,173.300000,0.000000,41.620000,173.300000,86.000000,2217.000000,795.000000,0.000000e+00,NaN,NaN,NaN,NaN,2.000000,0.000000,0.000000,0.000000,0.000000
75%,2019.000000,102.000000,110.000000,194.000000,0.000000,44.060000,194.000000,182.000000,3160.000000,1013.000000,0.000000e+00,NaN,NaN,NaN,NaN,4.000000,1.000000,0.000000,0.000000,0.000000
max,2022.000000,1006.000000,207.000000,305.000000,50.000000,82.880000,305.000000,538.000000,6104.000000,1812.000000,3.740700e+07,NaN,NaN,NaN,NaN,5.000000,1.000000,1.000000,1.000000,1.000000


In [110]:
df_small = merged_df[['distance','result', 'ProfileScore:','Vert. meters:', 'Startlist quality score:','Avg. speed winner:']]
df_big = merged_df.drop(columns=['distance', 'ProfileScore:','Vert. meters:', 'Startlist quality score:','Avg. speed winner:'])

In [103]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

   # DISTANCE PIPE
distance_min = 0.6
distance_max = 305
distance_transformer = FunctionTransformer(lambda p: (p - distance_min) /
     (distance_max - distance_min))

# RESULT PIPE
result_min = 1
result_max = 207
result_transformer = FunctionTransformer(lambda p: (p - result_min) /
     (result_max - result_min))

# PROFILE PIPE
profile_min = 11
profile_max = 538
profile_transformer = FunctionTransformer(lambda p: (p - profile_min) /
     (profile_max - profile_min))

# VERT PIPE
vert_min = 0
vert_max = 6104
vert_transformer = FunctionTransformer(lambda p: (p - vert_min) /
     (vert_max - vert_min))

# STARTLIST PIPE
startlist_min = 0
startlist_max = 1812
startlist_transformer = FunctionTransformer(lambda p: (p - startlist_min) /
     (startlist_max - startlist_min))

# STARTLIST PIPE
speed_min = 0
speed_max = 82
speed_transformer = FunctionTransformer(lambda p: (p - speed_min) /
     (speed_max - speed_min))


In [104]:
        # COMBINED PREPROCESSOR
final_preprocessor = ColumnTransformer(
            [
                ("distance_transformer", distance_transformer, ["distance"]),
                ("result_transformer", result_transformer, ["result"]),
                ("profile_transformer", profile_transformer, ["ProfileScore:"]),
                ("vert_transformer", vert_transformer, ["Vert. meters:"]),
                ("startlist_transformer", startlist_transformer, ["Startlist quality score:"]),
                ("speed_transformer", speed_transformer, ["Avg. speed winner:"]),
            ], remainder="drop",
            n_jobs=-1,
        )

In [117]:
scaled_df=pd.DataFrame(final_preprocessor.fit_transform(df_small), columns=df_small.columns).rename(columns={'result':'result_scaled'})
scaled_df.merge(df_big, left_index=True, right_index=True).columns

Index(['distance', 'result_scaled', 'ProfileScore:', 'Vert. meters:',
       'Startlist quality score:', 'Avg. speed winner:', 'name', 'year',
       'type', 'date', 'result', 'gc', 'icon', 'race_ref', 'race_name',
       'race_detail', 'race_rank', 'points', 'Distance:', 'Parcours type:',
       'Won how: ', 'adjusted_points', 'Race category:', 'Points scale:',
       'Arrival:', 'Won how:', 'parcours_type_num', 'gt_binary', 'key',
       'result_bin', 'types_bin', 'icon_bin'],
      dtype='object')

In [21]:
merged_df.isna().sum().sum()

4085849

In [22]:
merged_df.head()

,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,Race category:,Points scale:,Arrival:,Won how:,parcours_type_num,gt_binary,key,result_bin,types_bin,icon_bin
1,andrey-amador,2020,etappe,2020-09-16,117,79.0,stage,race/tour-de-france/2020/stage-17,tour-de-france,Stage 17 - Grenoble › Méribel - Col de la Loze,...,NaN,NaN,NaN,NaN,5.0,1,andrey-amador-2020-tour-de-france,0.0,0,0
2,richard-carapaz,2020,etappe,2020-09-16,11,13.0,stage,race/tour-de-france/2020/stage-17,tour-de-france,Stage 17 - Grenoble › Méribel - Col de la Loze,...,NaN,NaN,NaN,NaN,5.0,1,richard-carapaz-2020-tour-de-france,1.0,0,0
3,jonathan-castroviejo,2020,etappe,2020-09-16,116,40.0,stage,race/tour-de-france/2020/stage-17,tour-de-france,Stage 17 - Grenoble › Méribel - Col de la Loze,...,NaN,NaN,NaN,NaN,5.0,1,jonathan-castroviejo-2020-tour-de-france,0.0,0,0
4,michal-kwiatkowski,2020,etappe,2020-09-16,130,37.0,stage,race/tour-de-france/2020/stage-17,tour-de-france,Stage 17 - Grenoble › Méribel - Col de la Loze,...,NaN,NaN,NaN,NaN,5.0,1,michal-kwiatkowski-2020-tour-de-france,0.0,0,0
5,luke-rowe,2020,etappe,2020-09-16,131,133.0,stage,race/tour-de-france/2020/stage-17,tour-de-france,Stage 17 - Grenoble › Méribel - Col de la Loze,...,NaN,NaN,NaN,NaN,5.0,1,luke-rowe-2020-tour-de-france,0.0,0,0


In [23]:
#split to predict later
merged_df_predict = merged_df[merged_df['year'] == 2022]
merged_df_train = merged_df[(merged_df['year'] > 2017) & (merged_df['year'] != 2022)]

In [24]:
#riders to predict
riders_predict = merged_df_predict[(merged_df_predict['race_name']=='tour-de-france')| (merged_df_predict['race_name']=='vuelta-a-espana')| (merged_df_predict['race_name']=='giro-d-italia')][['name', 'year', 'race_name']]
riders_predict = riders_predict[riders_predict['year'] == 2022].drop_duplicates().reset_index(drop=True)
riders_predict

,name,year,race_name
0,tadej-pogacar,2022,tour-de-france
1,mikkel-bjerg,2022,tour-de-france
2,rafal-majka,2022,tour-de-france
3,brandon-mcnulty,2022,tour-de-france
4,marc-soler,2022,tour-de-france
...,...,...,...
347,sergio-samitier,2022,giro-d-italia
348,filippo-fiorelli,2022,giro-d-italia
349,jakub-mareczko,2022,giro-d-italia
350,miguel-angel-lopez,2022,giro-d-italia


In [25]:
#riders to train on
riders = merged_df_train[(merged_df_train['race_name']=='tour-de-france')| (merged_df_train['race_name']=='vuelta-a-espana')| (merged_df_train['race_name']=='giro-d-italia')][['name', 'year', 'race_name']]
riders = riders[riders['year'] != 2022].drop_duplicates().reset_index(drop=True).sort_values(by='year', ascending=False)
riders

,name,year,race_name
1997,frederik-frison,2021,vuelta-a-espana
1628,jack-haig,2021,tour-de-france
1634,bauke-mollema,2021,giro-d-italia
1633,vincenzo-nibali,2021,giro-d-italia
1632,dan-martin,2021,giro-d-italia
...,...,...,...
1278,tony-gallopin,2018,vuelta-a-espana
1279,vincenzo-nibali,2018,vuelta-a-espana
1280,gorka-izagirre,2018,vuelta-a-espana
1281,ion-izagirre,2018,vuelta-a-espana


In [9]:
# #split riders in train/test randomly
# train, test = train_test_split(riders, test_size=0.2, random_state=42)

# #split rediers in train/test based on years
# train = riders[riders['year']<2021]
# test = riders[riders['year']>=2021]

train = riders.copy()
test = riders_predict.copy()

len(test)/len(train)

0.17617617617617617

In [10]:
X_encoder, X_decoder = get_scaler(80, merged_df, riders)


 dropped nan for ('krists-neilands', 2020, 'tour-de-france')

 dropped nan for ('marco-frapporti', 2020, 'giro-d-italia')

 dropped nan for ('jacopo-mosca', 2020, 'giro-d-italia')

 dropped nan for ('jacopo-mosca', 2019, 'vuelta-a-espana')

 dropped nan for ('luis-angel-mate', 2019, 'vuelta-a-espana')


In [11]:
X_encoder_train, X_decoder_train, y_decoder_train = get_sequences(80, merged_df, train, X_encoder, X_decoder)
X_encoder_test, X_decoder_test, y_decoder_test = get_sequences(80, merged_df, test, X_encoder, X_decoder)


 Getting sequence for ('frederik-frison', 2021, 'vuelta-a-espana')

 Getting sequence for ('jack-haig', 2021, 'tour-de-france')

 Getting sequence for ('bauke-mollema', 2021, 'giro-d-italia')

 Getting sequence for ('vincenzo-nibali', 2021, 'giro-d-italia')

 Getting sequence for ('dan-martin', 2021, 'giro-d-italia')

 Getting sequence for ('jonathan-castroviejo', 2021, 'giro-d-italia')

 Getting sequence for ('davide-formolo', 2021, 'giro-d-italia')

 Getting sequence for ('marc-soler', 2021, 'tour-de-france')

 Getting sequence for ('robert-gesink', 2021, 'tour-de-france')

 Getting sequence for ('dries-de-bondt', 2021, 'giro-d-italia')

 Getting sequence for ('caleb-ewan', 2021, 'tour-de-france')

 Getting sequence for ('bryan-coquard', 2021, 'tour-de-france')

 Getting sequence for ('loic-vliegen', 2021, 'tour-de-france')

 Getting sequence for ('nickolas-dlamini', 2021, 'tour-de-france')

 Getting sequence for ('stefan-de-bod', 2021, 'tour-de-france')

 Getting sequence for ('jas

In [12]:
X_encoder_train.shape, X_decoder_train.shape, y_decoder_train.shape


((1995, 80, 15), (1995, 21, 5), (1995, 21, 1))

In [231]:
X_encoder_test.shape, X_decoder_test.shape, y_decoder_test.shape


((352, 80, 15), (352, 21, 5), (352, 21, 1))

In [232]:
np.unique(y_decoder_train, return_counts=True)[1]/32917

array([0.11443935, 1.        , 0.15830726])

In [233]:
from cycling_manager.sequences import *
from cycling_manager.preprocess import *
from cycling_manager.model import *

In [263]:
model = combine_model(X_encoder_train, X_decoder_train)

model = compile_model(model)


Combined model...
\Compile model...


In [264]:
model, history = train_model(model, X_encoder_train, X_decoder_train, y_decoder_train)

Epoch 1/1000
11/11 [==============================] - 10s 266ms/step - loss: 101.3756 - accuracy: 0.1230 - val_loss: 66.3950 - val_accuracy: 0.1278
Epoch 2/1000
11/11 [==============================] - 1s 65ms/step - loss: 101.3459 - accuracy: 0.1229 - val_loss: 66.4002 - val_accuracy: 0.1278
Epoch 3/1000
11/11 [==============================] - 1s 65ms/step - loss: 101.3378 - accuracy: 0.1229 - val_loss: 66.4014 - val_accuracy: 0.1278
Epoch 4/1000
11/11 [==============================] - 1s 65ms/step - loss: 101.3356 - accuracy: 0.1229 - val_loss: 66.4016 - val_accuracy: 0.1278
Epoch 5/1000
11/11 [==============================] - 1s 67ms/step - loss: 101.3348 - accuracy: 0.1229 - val_loss: 66.4017 - val_accuracy: 0.1278
Epoch 6/1000
11/11 [==============================] - 1s 66ms/step - loss: 101.3345 - accuracy: 0.1229 - val_loss: 66.4018 - val_accuracy: 0.1278
Epoch 7/1000
11/11 [==============================] - 1s 72ms/step - loss: 101.3343 - accuracy: 0.1229 - val_loss: 66.4018

In [265]:
model.evaluate([X_encoder_test, X_decoder_test], y_decoder_test)

11/11 [==============================] - 0s 8ms/step - loss: 85.4516 - accuracy: 0.1136


[85.45160675048828, 0.11363636702299118]

In [248]:
preds = model.predict([X_encoder_test, X_decoder_test])

pred_ls = []

for pred, rid in zip(preds, riders_predict.values):
    pred_ls.append(np.append(pred.round(7), rid))

11/11 [==============================] - 3s 8ms/step


In [249]:
pred_df = pd.DataFrame(pred_ls).replace(1.000, np.nan)

In [250]:
pred_df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.682626,0.730118,0.727142,0.728404,0.718507,0.760474,0.775399,0.754974,0.767529,0.769384,...,0.747234,0.771534,0.777406,0.786003,0.748377,0.716176,0.709863,tadej-pogacar,2022,tour-de-france
1,0.678330,0.724978,0.722238,0.723741,0.713868,0.756744,0.771461,0.751183,0.763138,0.765441,...,0.743284,0.767185,0.773136,0.782022,0.744522,0.711970,0.705685,mikkel-bjerg,2022,tour-de-france
2,0.683558,0.730137,0.727430,0.728766,0.719018,0.761159,0.775824,0.755670,0.767878,0.769874,...,0.747945,0.771882,0.000000,0.000000,0.000000,0.000000,0.000000,rafal-majka,2022,tour-de-france
3,0.682623,0.729221,0.726438,0.728125,0.718363,0.760006,0.775219,0.754497,0.767188,0.769269,...,0.746749,0.771197,0.777075,0.785727,0.747893,0.716173,0.709873,brandon-mcnulty,2022,tour-de-france
4,0.680841,0.727455,0.724731,0.726166,0.716345,0.758873,0.773537,0.753349,0.765371,0.767555,...,0.745557,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,marc-soler,2022,tour-de-france
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,0.669493,0.591430,0.643757,0.709758,0.655462,0.657574,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,sergio-samitier,2022,giro-d-italia
348,0.701651,0.621271,0.672318,0.739037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,filippo-fiorelli,2022,giro-d-italia
349,0.672680,0.594016,0.646844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,jakub-mareczko,2022,giro-d-italia
350,0.705954,0.625054,0.675683,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,miguel-angel-lopez,2022,giro-d-italia


In [251]:
pred_df[23].unique()

array(['tour-de-france', 'giro-d-italia'], dtype=object)

In [256]:
pred_tdf = pred_df[pred_df[23]=='giro-d-italia'].drop(columns=[22,23])


In [266]:
pred_tdf[[21]].merge(pd.DataFrame(pred_tdf.mean(axis=1)), left_index=True, right_index=True).sort_values(by=0, ascending=False).head(70)

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_60276/997402418.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pred_tdf[[21]].merge(pd.DataFrame(pred_tdf.mean(axis=1)), left_index=True, right_index=True).sort_values(by=0, ascending=False).head(70)


,21,0
188,magnus-cort-nielsen,0.709270
192,richard-carapaz,0.708556
195,pavel-sivakov,0.708436
304,lucas-hamilton,0.708414
218,wout-poels,0.708101
...,...,...
227,giovanni-aleotti,0.703554
319,rui-costa,0.703523
265,jan-hirt,0.703517
202,lawrence-naesen,0.703446


In [258]:
arr = np.array([[0,2,4],[6,8,10]]) 
app_arr = np.append(arr, [13,15,17]) 

In [262]:
app_arr.reshape(3, 1,3)

array([[[ 0,  2,  4]],

       [[ 6,  8, 10]],

       [[13, 15, 17]]])